In [34]:
#Current Problem

#task split function creates a chain of tasks, but it should be a hierarchy

#provide taskSolver most helpful results of task childs (sometimes context is polutet)

#results are added in the wrong order in dummySolver

#an return all usefull information in this text for this task( and the context of the taskpath)

# minor:Check if task is solved correctly could be done with fewer tokens

# https://www.reddit.com/r/AutoGPT/comments/12k4al2/found_the_winning_sauce/

#IDEA Make the VirtualFileOperations so they contain a message what has been done. This could be used to create a log of the process that can be used to create the TaskResultMessage

# use caching of LLM calls

#make tooluse an process:
    # what you wane do? i suggest tools?
    # what suggested tool you wane use?
    # heres a description how exacly to use that tool
    # execute the tool

#fundamental a workingdir with write and read access and a obligatory paginated output is needed
#solver could have verry simple read commands like 
#READ FILENAME PAGE
#WRITE FILENAME PAGE
#LIST FILENAMES

#TODO READ FIle with sourounding should indicate where surounding is

In [35]:
#we gone need many types of BrainFunctions

#Tasks
    #tasks have subtasks
    #if a task couldn't be solved, it can be retried later
    #tasks should consist of descriptions results and parameters

#taskpath 
    #Path from Objectiv Task to current Task. Creates a great context for the current task.

#Atomic Tasks are tasks with no subtasks

#Central control
#    decides what to do next.
#       Split Tast
#       Solve Task
#       Retry Task
#       Improve Task Solution

#TaskSplitter Task->List[Task] (simpler tasks)
#TaskSolver Task->Result + Optional List[Task] (new tasks encountered during solving, debugging, etc.)
#Task Duplicate Detector List[Task]->List[Task]
#Task Merger List[Task]->Task
#Task Selector 
#Function tath gathers all the results that can be helpfull to solve a task
#asign solved task to other tasks that could benefit from the results
#ask for most complex task and split it into subtasks


#JSON Stuffer Takes result of Brainfunction and stuffs it into a JSON string
#retries with "doesnt look like correct JSNO jet" until its successfull parsed into object

In [4]:
import json
import re
from collections import deque
from typing import Dict, Iterable, List, Set, Optional, Any

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from openai import InvalidRequestError
from pydantic import BaseModel, Field
from langchain.chains.base import Chain

from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

import os
from langchain.chat_models import PromptLayerChatOpenAI
from langchain.schema import HumanMessage
import sqlite3

class VirtualFile:
    def __init__(self, content, page_size):
        self.pages = [content[i:i+page_size] for i in range(0, len(content), page_size)]


class VirtualFileSystem:
    def __init__(self, page_size=2000):
        self.virtual_files = {}
        self.page_size = page_size

    def dump_all(self, dump_directory="dump"):
        if not os.path.exists(dump_directory):
            os.makedirs(dump_directory)

        for file_name in self.virtual_files:
            file_path = os.path.join(dump_directory, file_name)
            with open(file_path, 'w') as f:
                for page in self.virtual_files[file_name].pages:
                    f.write(page)
        return f"All virtual files saved to '{dump_directory}' directory."


    def create_file(self, file_name, content):
        if file_name in self.virtual_files:
            return f"Error: File '{file_name}' already exists."
        self.virtual_files[file_name] = VirtualFile(content, self.page_size)
        return f"File '{file_name}' created."

    def read_file(self, file_name, page, include_surrounding=False, surrounding_chars=100):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        if page >= len(self.virtual_files[file_name].pages):
            return f"Error: Invalid page: {page}"
        
        content = self.virtual_files[file_name].pages[page]

        if include_surrounding:
            prev_page = self.virtual_files[file_name].pages[page - 1] if page > 0 else ""
            next_page = self.virtual_files[file_name].pages[page + 1] if page < len(self.virtual_files[file_name].pages) - 1 else ""

            prev_content = prev_page[-surrounding_chars:] if prev_page else ""
            next_content = next_page[:surrounding_chars] if next_page else ""

            content = f"{prev_content}{content}{next_content}"

        return content

    def update_file(self, file_name, page, new_content):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        if page >= len(self.virtual_files[file_name].pages):
            return f"Error: Invalid page: {page}"
        self.virtual_files[file_name].pages[page] = new_content
        self.reorganize_pages(file_name)
        return f"File '{file_name}' updated at page {page}."

    def save_to_disk(self, file_name, file_path=None):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        if not file_path:
            file_path = file_name
        with open(file_path, 'w') as f:
            for page in self.virtual_files[file_name].pages:
                f.write(page)
        return f"File '{file_name}' saved to disk at '{file_path}'."
    
    def load_from_disk(self, file_path, file_name=None):
        if not os.path.exists(file_path):
            return f"Error: No such file on disk: '{file_path}'"
        if not file_name:
            file_name = os.path.basename(file_path)
        if file_name in self.virtual_files:
            return f"Error: File already exists in VFS: '{file_name}'"
        with open(file_path, 'r') as f:
            content = f.read()
        self.virtual_files[file_name] = VirtualFile(content, self.page_size)
        return f"File '{file_path}' loaded from disk into VFS as '{file_name}'."

    def list_files(self):
        output = "Current available Files:\n"
        for file_name in self.virtual_files.keys():
            file = self.virtual_files[file_name]
            output += f"File '{file_name}' has {len(file.pages)} pages and a total size of {sum(len(page) for page in file.pages)} characters.\n"
        return output+"\n"

    def delete_file(self, file_name):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        del self.virtual_files[file_name]
        return f"File '{file_name}' deleted."

    def append_to_file(self, file_name, content):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        self.virtual_files[file_name].pages.extend(
            [content[i:i+self.page_size] for i in range(0, len(content), self.page_size)]
        )
        return f"Content appended to file '{file_name}'."

    def rename_file(self, old_name, new_name):
        if old_name not in self.virtual_files:
            return f"Error: No such file: '{old_name}'"
        if new_name in self.virtual_files:
            return f"Error: File already exists: '{new_name}'"
        self.virtual_files[new_name] = self.virtual_files.pop(old_name)
        return f"File '{old_name}' renamed to '{new_name}'."

    def get_file_info(self, file_name):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        file = self.virtual_files[file_name]
        return f"File '{file_name}' has {len(file.pages)} pages and a total size of {sum(len(page) for page in file.pages)} characters."

    def reorganize_pages(self, file_name):
        if file_name not in self.virtual_files:
            return f"Error: No such file: '{file_name}'"
        file = self.virtual_files[file_name]
        content = "".join(file.pages)
        file.pages = [content[i:i+self.page_size] for i in range(0, len(content), self.page_size)]
        return f"Pages reorganized for file '{file_name}'."
    

    def execute_command(self, command):
        command_parts = command.split()
        if not command_parts:
            return "Error: Empty command."

        command_name = command_parts[0].upper()
        args = command_parts[1:]

        if command_name == "CREATE_FILE":
            if len(args) == 1:
                return self.create_file(args[0], "")
            if len(args) < 2:
                return "Error: CREATE_FILE requires at least 2 arguments."
            return self.create_file(args[0], " ".join(args[1:]))

        elif command_name == "READ_FILE":
            if len(args) < 2:
                return "Error: READ_FILE requires at least 2 arguments."
            include_surrounding = "INCLUDE_SURROUNDING" in args
            if include_surrounding:
                args.remove("INCLUDE_SURROUNDING")
            surrounding_chars_index = next((i for i, x in enumerate(args) if x == "SURROUNDING_CHARS"), None)
            if surrounding_chars_index is not None and surrounding_chars_index + 1 < len(args):
                surrounding_chars = int(args[surrounding_chars_index + 1])
                args = args[:surrounding_chars_index] + args[surrounding_chars_index + 2:]
            else:
                surrounding_chars = 100
            return self.read_file(args[0], int(args[1]), include_surrounding, surrounding_chars)

        elif command_name == "UPDATE_FILE":
            if len(args) < 3:
                return "Error: UPDATE_FILE requires at least 3 arguments."
            return self.update_file(args[0], int(args[1]), " ".join(args[2:]))

        elif command_name == "SAVE_TO_DISK":
            if len(args) < 1:
                return "Error: SAVE_TO_DISK requires at least 1 argument."
            return self.save_to_disk(args[0], args[1] if len(args) > 1 else None)

        elif command_name == "LIST_FILES":
            return self.list_files()

        elif command_name == "DELETE_FILE":
            if len(args) < 1:
                return "Error: DELETE_FILE requires at least 1 argument."
            return self.delete_file(args[0])

        elif command_name == "APPEND_TO_FILE":
            if len(args) < 2:
                return "Error: APPEND_TO_FILE requires at least 2 arguments."
            return self.append_to_file(args[0], " ".join(args[1:]))

        elif command_name == "RENAME_FILE":
            if len(args) < 2:
                return "Error: RENAME_FILE requires at least 2 arguments."
            return self.rename_file(args[0], args[1])

        elif command_name == "GET_FILE_INFO":
            if len(args) < 1:
                return "Error: GET_FILE_INFO requires at least 1 argument."
            return self.get_file_info(args[0])

        elif command_name == "REORGANIZE_PAGES":
            if len(args) < 1:
                return "Error: REORGANIZE_PAGES requires at least 1 argument."
            return self.reorganize_pages(args[0])

        elif command_name == "DUMP_ALL":
            return self.dump_all(args[0] if args else None)

        else:
            return f"Error: Unknown command '{command_name}'."

class Task:
    def __init__(self, TaskDescription: str = "dscr", resultDescription:str = "not solved", Solved:bool = False):
        self.Description:str = TaskDescription
        self.ResultDescription:str = resultDescription
        self.Solved:bool = Solved
        self.ChildTasks:List[Task] = []
        self.ParentTasks:List[Task] = []
        
    def __str__(self)->str:
        if(self.Solved):
            return "Solved["+self.Description+"]>>"+self.ResultDescription
        else:
            return "Unsolved["+self.Description+"]>>"+self.ResultDescription
        
    def get_recusively_all_subtasks(self, visited: Set['Task'] = None) -> List['Task']:
        if visited is None:
            visited = set()
        if self in visited:
            return []
        all_tasks = []  # Add the current task to the list
        for subtask in self.ChildTasks:
            all_tasks.append(subtask)
            all_tasks +=subtask.get_recusively_all_subtasks(visited)  # Recursively get all subtasks of the subtask
        return all_tasks

    def display(self, indent_level: int = 0) -> None:
        indent = '  ' * indent_level  # Modify the number of spaces in the quotes to adjust indentation
        if(self.Solved):
            print(indent+"Solved["+self.Description+"]>>"+self.ResultDescription.replace("\n",""))
        else:
            print(indent+"Unsolved["+self.Description+"]>>"+self.ResultDescription)
        for subtask in self.ChildTasks:
            subtask.display(indent_level + 1)
    
    def collect_parent_descriptions(self, visited: Set['Task'] = None) -> List[str]:
        if visited is None:
            visited = set()
        if self in visited:
            return []
        visited.add(self)
        descriptions = []
        for parent_task in self.ParentTasks:
            descriptions.append(parent_task.Description)
            descriptions.extend(parent_task.collect_parent_descriptions(visited))
        return descriptions
    

    def is_atomic(self)->bool:
        return len(self.ChildTasks) == 0
    
    def ready_to_solve(self)->bool:
            return all(map(lambda x: x.Solved, self.ChildTasks)) #all([]) is True
        
    def add_child(self, child: 'Task') -> None:
        if child in self.ChildTasks:
            return
        self.ChildTasks.append(child)
        child.ParentTasks.append(self)

    def add_childs(self, childs: List['Task']) -> None:
        for child in childs:
            self.add_child(child)

class Taskpool(List[Task]):    
    def __str__(self)->str:
        return ",".join(map(str, self.tasks))
    
    def CreateContextSummery(self)->str:
        return f"/n".join(map(lambda x: x.TaskDescription, self.tasks))
    
    def get_task_ready_to_solve(self)->Task:
        for task in self:
            if(task.Solved):
                continue
            if(task.ready_to_solve()):
                return task
        raise Exception("No task ready to solve")
    
    def get_nummerated_task_List(self) -> str:
        formatted_tasks = []
        for index, task in enumerate(self):
            formatted_tasks.append(f"{index}:{task.Description}")
        return "\n".join(formatted_tasks)
    
class CachedChat:
    def __init__(self):
        self.conn = None
        self.chat = PromptLayerChatOpenAI(model_name="gpt-4",temperature=0.2, request_timeout=380)
        # self.chat = PromptLayerChatOpenAI(temperature=0.2, request_timeout=180)

    def __call__(self, messages, stop=None):

        return AIMessage(
            content=self.run_chat(messages, stop)
        )
    
    def establish_connection(self):
        self.conn = sqlite3.connect('cache.db')

    def check_cache(self, query):
        cursor = self.conn.cursor()
        cursor.execute("SELECT response FROM cache WHERE query=?", (query,))
        result = cursor.fetchone()
        return result[0] if result else None

    def store_cache(self, query, response):
        cursor = self.conn.cursor()
        cursor.execute("INSERT INTO cache(query, response) VALUES (?, ?)", (query, response))
        self.conn.commit()

    def create_cache_table(self):
        cursor = self.conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS cache
                        (query TEXT PRIMARY KEY, response TEXT)''')
        self.conn.commit()

    def run_chat(self, messages, stop) -> str:
        query = str((messages, stop))
        self.establish_connection()
        self.create_cache_table()
        cached_response = self.check_cache(query)

        if cached_response:
            return cached_response

        chat_response = self.chat(messages, stop)
        self.store_cache(query, chat_response.content)

        return chat_response.content

class Agent():
    def __init__(self, Objectiv: str):
        self.Objectiv:Task = Task(Objectiv)
        self.Taskpool = Taskpool()
        self.Taskpool.append(self.Objectiv)
        # self.chat = ChatOpenAI(temperature=0)
        #self.chat = PromptLayerChatOpenAI(temperature=0.2)
        self.chat=CachedChat()

        self.VirtualFileSystem=VirtualFileSystem(page_size=2000)
        self.SelfDescription = """Alright, so you're part of an autonomous agent that utilizes a sophisticated tool called ChatGPT to divide complex tasks into smaller, more manageable ones, and than solves them. Although ChatGPT is quite intelligent, its context awareness is limited, so we have to employ a paging system to assist it in retaining and processing data. Our communication among team members is facilitated through file sharing. We extract information from files and store the results in files as well. To ensure the timely and efficient execution of the current task, our task splitter coordinates the data flow, guaranteeing that we have all the necessary preparations at our disposal."""
        self.command_documentation = """Command Documentation
CALL CREATE_FILE <file_name>
<content>
ENDCALL
- Creates a new file with the given name and content.

CALL READ_FILE <file_name> <page> [INCLUDE_SURROUNDING] [SURROUNDING_CHARS <num_chars>]
ENDCALL
- Reads the specified page of the file. Optionally, include surrounding content from adjacent pages with the specified number of characters.

CALL UPDATE_FILE <file_name> <page>
<new_content>
ENDCALL
- Updates the specified page of the file with the new content.

CALL LIST_FILES
ENDCALL
- Lists all the files in the FS.

CALL DELETE_FILE <file_name>
ENDCALL
- Deletes the specified file from the FS.

CALL APPEND_TO_FILE <file_name>
<content>
ENDCALL
- Appends the given content to the specified file.

CALL RENAME_FILE <old_name> <new_name>
ENDCALL
- Renames the specified file to the new name.

CALL GET_FILE_INFO <file_name>
ENDCALL
- Retrieves information about the specified file, such as the number of pages and total size.

CALL FINISH
<Summary of files created and their contents as hints for Parent Tasks>
ENDCALL
- Returns the summary of files created and their contents as hints for Parent Tasks. This ends the solving of the task. Call this when you are done.

Example:
CALL CREATE_FILE test.txt
This is the content of the file
ENDCALL

The first Page in a File is Page number 0.

Only ever create one command. You will be provided the results of an command and than are prompted again to create a new Command.
Its not your task to simulate the command execution! Dont create Returnmessages for Commands.

Suggested Workflow:
Get current list of files.
Read Files that could be usefull, if any
Create new File with your solution
finish with FINISH command

"""
    def extract_json(self,text):
        # Find a JSON object or array pattern in the text
        pattern = r'({.*?}|\[.*?\])'
        match = re.search(pattern, text, re.DOTALL)

        # Check if a match is found
        if match:
            json_str = match.group()
            try:
                # Parse the JSON string into a Python object
                json_obj = json.loads(json_str)
                return json_obj
            except json.JSONDecodeError:
                print("Invalid JSON detected.")
                return None
        else:
            print("No JSON found in the text.")
            return None

    def should_be_split(self, task: Task) -> bool:
        #test if task is atomic if no throw exception
        if len(task.ChildTasks) != 0:
            raise ValueError("Task is not atomic")
        
        print("Now in should_be_slpit:" + task.Description)

        
       

        messages = []
        messages.append(SystemMessage(content=
                                      "You are an AI that determines if a task could be split into multiple simpler tasks. Return 'True' if it could be split and 'False' otherwise. Dont be chaty."))
        
        messages.append(HumanMessage(content=
                                     f"Return 'True' if it could be split and 'False' otherwise. Task description: {task.Description}"))
        
        max_retries = 3
        for _ in range(max_retries):
            response = self.chat(messages).content
            clean_response = response.strip().lower()

            if 'true' in clean_response:
                return True
            elif 'false' in clean_response:
                return False
            else:
                messages.append(HumanMessage(content="Please provide a valid response with either 'True' or 'False'."))
        raise ValueError("AI response does not contain a valid boolean string after multiple retries: " + response)

    def SplitTask(self, task: Task, verbose: bool = True) -> List[Task]:
        print("Now in SplitTask:" + task.Description)

        messages = []

        messages.append(SystemMessage(content=f"""{self.SelfDescription}

As a Task Splitting AI, your objective is to analyze the main task provided and create smaller, independent, and manageable subtasks that contribute to its successful completion. Create a hierarchical view, prioritizing tasks that need to be completed first. Ensure that the subtasks do not expand the scope of the main task and follow it literally. Keep in mind the limited context problem faced by language models like ChatGPT when processing and generating text that exceeds their maximum token limit.

In each subtask:
- Instruct the use of the FileSystem to store results.
- Return information about the created or modified files, along with a brief description of their contents.

Clearly define the expected results and file names for each task to be produced. You cannot create variables, only files. Always instruct each task to call the FINISH command at the end.

The TaskSolver has the following commands available:
CREATE_FILE, READ_FILE, UPDATE_FILE, LIST_FILES, DELETE_FILE, APPEND_TO_FILE, RENAME_FILE, GET_FILE_INFO, and FINISH.

Do not number your tasks, but list them in the order they should be executed.

Well working example of an subtask has been:
The File FinalAGI.py is loaded for you. Simply call the ReadFile Command to read each Page of that file. After you have read a page, apend a summary to the file summary.txt. After you have read all pages, call the FINISH command and write a short summary of your findings.

"""))


        # Check if the task has any parent tasks
        # parent_descriptions = task.collect_parent_descriptions()
        # if parent_descriptions:
        #     # Include the descriptions of the parent tasks in the task description
        #     task_description = f" Maintask: {task.Description}. The context of the parent tasks is: {', '.join(parent_descriptions)}"
        #     task_description += "\n Use this context to split the main task into smaller, independent, and more manageable subtasks."
        #     messages.append(HumanMessage(content=task_description))
        # else:
        #     messages.append(HumanMessage(content=MainPromp+ """ Main task to split: """ + task.Description))

        # Call OpenAI

        messages.append(HumanMessage(content="""Main task to split: """ + task.Description))
        generation = self.chat(messages)

        print("Splitted Tasks before JSON packing: " + generation.content)


        #refinment step
        messages.append(generation)
        messages.append(HumanMessage(content="""Have you followed ale the rules descriped to create this task list? How could better Subtasks look?"""))
        generation = self.chat(messages)
        messages.append(generation)

        #human feedback step
        print(generation.content)
        print("What do you want to change at this plan? Simply press Enter if happy")
        while True:
            feedback = input()
            if feedback.strip() == "":
                break
            else:
                messages.append(HumanMessage(content = f"The Operator wants this changes to the plan: {feedback}"))
                generation = self.chat(messages)
                messages.append(generation)
                print(generation.content)
                print("What do you want to change at this plan? Simply press Enter if happy")

        # JSON Packaging step
        messages.append(generation)
        messages.append(HumanMessage(content="""Now put this Tasks with their complete description in full detail in an element of a JSON Array like ["Complete Description of first Task....", "Complete description of second Task...."]."""))

        # Call OpenAI
        JSONSTR = self.chat(messages).content
        print("Splitted Tasks as JSON " + JSONSTR)

        JSON = self.extract_json(JSONSTR)

        Tasks = []

        for subtaskstr in JSON:
            Tasks.append(Task(TaskDescription=str(subtaskstr)))

        # Link the tasks as child tasks in a sequential manner, each task is a child task of the task before it
        Tasks.reverse()  # Last Task depends on Tasks before it so he is the first with a child
        task.add_child(Tasks[0])
        for i in range(len(Tasks) - 1):
            Tasks[i].add_child(Tasks[i + 1])

        return Tasks
    
    def SolveAtomicTask(self, task: Task, max_retries: int = 5, verbose: bool = True) -> Task:
        ContextHeader = []
        ContextHeader.append(SystemMessage(content=f"""{self.SelfDescription}
You are an AI designed to solve tasks by understanding the requirements, performing necessary file operations, and delivering the results. Stay focused on the task and adhere to the command documentation. Save the task results in files and provide a brief message indicating the created files and a concise description of their contents. Once completed, use the FINISH command."""))

        # # Include Parent Tasks in the task description
        # parent_descriptions = task.collect_parent_descriptions()
        # parent_descriptions.reverse() # Reverse the order of the parent descriptions so that the 'nearest' parent is last to be included in the task description (most relevant to the current task)
        # if parent_descriptions:
        #     task_description = f"Current Task: {task.Description}. \n\n This Current Task will be used to solve the parent tasks. Keep that in mind while solving it, so its solved in a way, its usefull in the parent Tasks.\n\n Parent Tasks: {os.linesep.join(parent_descriptions)}"

        #     finalMessages.append(HumanMessage(content=task_description))

        # childs = task.get_recusively_all_subtasks()
        # if len(childs) > 0:
        #     task_description = f"Current Task: {task.Description}. The Current Task has Subtasks that are solved. The results and descriptions of the child tasks are:\n\n" + '\n\n'.join(map(lambda t: (t.Description + ": " + t.ResultDescription), childs))
        #     task_description += "\n Use these results to solve the current task."
        #     finalMessages.append(HumanMessage(content=task_description))

        ContextHeader.append(HumanMessage(content=self.command_documentation))

        ContextHeader.append(HumanMessage(content=self.VirtualFileSystem.list_files()))
        ContextHeader.append(HumanMessage(content=f"Current Task: {task.Description}"))

        workingLog = []
        for i in range(200): #max 10 commands
            workingLog.append(HumanMessage(content="We now make a thinking step, no commands can be send. Think step by step. What Command would be right next one? How is it used? You can only call one command at a time. No Piping is possible. This is just the planingphase!"))

            # call OpenAI for Chain of Thought
            toLong = True
            while toLong:
                try:
                    workingLog.append(self.chat(ContextHeader + workingLog))
                    toLong = False
                except InvalidRequestError:
                    workingLog=workingLog[1:] # remove one from working log and try again
                    print("Removed one message from working log and try again")
            

            # Call OpenAI for Command
            workingLog.append(HumanMessage(content="Now CALL the Command you thought of with the actual Contend. Remember you can only Call one command at a time. No Piping is possible."))
            
            toLong = True
            while toLong:
                try:
                    solutionMessage = self.chat(ContextHeader + workingLog, stop=["ENDCALL"])
                    toLong = False
                except InvalidRequestError:
                    workingLog=workingLog[1:] # remove one from working log and try again
                    print("Removed one message from working log and try again")

            
            solutionMessage.content+="\nENDCALL" #get the stop back in
            
            workingLog.append(solutionMessage)


            generation = solutionMessage.content.strip()
            #todo extract command from generation
            command_start = generation.find("CALL")
            command_end = generation.find("ENDCALL")
            if command_start != -1 and command_end != -1:
                command = generation[command_start + 4:command_end].strip()
                
                if "CALL FINISH" in generation:
                    task.ResultDescription = command.replace("FINISH\n", "").strip()
                    task.Solved = True
                    return task

                command_result = self.VirtualFileSystem.execute_command(command)

                if command_result:
                    workingLog.append(HumanMessage(content="Command Return:\n"+command_result))

            else:
                if verbose:
                    print(f"Retry {i + 1}: Command not found in generation.")
        return task
    
    def run(self):
        
        #hardcoded passes of possible splitting
        for i in range(0):
            currentTasks = Taskpool()
            currentTasks.extend(self.Taskpool) #create copy of Taskpool or else loop will be endless
            for task in currentTasks:
                if task.is_atomic():
                    #if self.should_be_split(task):
                    splitDone=True
                    print("Splitting Task")
                    newTasks = self.SplitTask(task)
                    self.Taskpool.extend(newTasks)
                    self.Objectiv.display()
                    print("Splitting Done")
        
        print(self.Taskpool.get_nummerated_task_List())

        while self.Objectiv.Solved==False:
            currentTask=self.Taskpool.get_task_ready_to_solve()
            #self.DummySolveAtomicTask(currentTask)
            self.SolveAtomicTask(currentTask)
            self.Objectiv.display()

# agent = Agent("Create a file containing a workflow how to write a novel")
# agent = Agent("Write a single HTML file in wich with JavaScript two AIs play Connect 4 against each other. The User only watches, and restarts.")

# agent=Agent("The File FinalAGI.py is loaded for you. Simply call the ReadFile Command to read each Page of that file. After you have read a page, apend a summary to the file summary.txt. After you have read all pages, call the FINISH command and write a short summary of your findings.")
# agent=Agent("Look at Page 5 of the file FinalAGI.py. Can you find a way to improve the code on that page? If so, replace the code on that page with your improved code. After you have replaced the code, create a file called changes.txt and write a short summary of your changes.")

agent.VirtualFileSystem.load_from_disk("FinalAGI.py")
agent.run()
agent.VirtualFileSystem.dump_all()

print(agent.Objectiv.ResultDescription)

0:The File FinalAGI.py is loaded for you. Simply call the ReadFile Command to read each Page of that file. After you have read a page, apend a summary to the file summary.txt. After you have read all pages, call the FINISH command and write a short summary of your findings.


KeyboardInterrupt: 